In [1]:
import h5py
import os
import time
import glob
import pandas as pd

import sys
import numpy as np
import gzip
import shutil
import pickle

In [78]:
def read_hdf5(BASEDIR, country, scenario):
    #find list with hdf5 files
    print("Find files. This might take a while")
    st0 = time.time()
    filename_list = glob.glob("%s/001_buildings.hdf5" %(BASEDIR)) 
    filename_list.extend(glob.glob("%s/**/_scen_*/001_buildings.hdf5" %(BASEDIR), recursive = True)) 
    
    print("%i files found in %4.2f sec"%(len(filename_list), time.time() - st0))
    
    if filename_list==[]:
        print("   --------- WARNING: There is no 001_buildings.hdf5 file for %s! ---------" % scenario)
        return []
    
    if 0==1:
        last_fn = sorted(glob.glob("%s/ZEN_RESULTS_*.pk"% BASEDIR))[-1]
    else:
        # Run through file list
        last_fn = None
        for filename in filename_list:
            #read hdf5 file
            print (filename)
            hdf5_f = h5py.File(filename,'r')

            # get data elements stored in hdf5 container
            item_names_hdf5 = hdf5_f.items()
            print("items stored in data container:")
            print(list(item_names_hdf5))
            
            # Extract simulation periods stored in data container
            year_list_hdf5 = []
            for curr_item_name in item_names_hdf5: 
                if curr_item_name[0][:3] == "BC_":
                    year_list_hdf5.append(int(curr_item_name[0][-4:]))
                
            print("Years stored in data container: %s" % str(year_list_hdf5))
            
            
            bca = hdf5_f["BuildingCategories"][()]             # building class file as Numpy recarray
            sector_idx_bca = bca["sector_idx"]
            is_res_bca = sector_idx_bca == 1
            
            t_indoor_thresholds = np.arange(13,25.5,0.5)
            thresh=t_indoor_thresholds.round(2)
            col_names =  ['Country', 'Year', 'Scenario', 'Weighted Average']
            col_names.extend(list(thresh))
            df_results  = pd.DataFrame(columns = col_names)
            
            for yr in year_list_hdf5: 
                print("Current yr: %i"%yr)
                st = time.time()
        
                key_bc = "BC_%i"%yr   # Building class file
                key_bssh = "BSSH_%i"%yr # building segment (space heating) file
                try:
                    bc = hdf5_f[key_bc][()]             # building class file as Numpy recarray
                    bssh = hdf5_f[key_bssh][()]         # building segment file as Numpy recarray
                except:
                    pass
                    continue
         
            
                bc_index = bc["index"]     #building class index of elements in bc dataset
                
                bc_index_bssh = bssh["building_classes_index"]            #building class index of elements in bssh dataset

                bca_index_bc = bc["building_categories_index"]          # vector that contains the building category index of each building class
                is_res_bc = is_res_bca[bca_index_bc - 1]                # vector that is true if building class is residential
                
                is_res_bssh = is_res_bc[bc_index_bssh - 1]      # vector that is true if building segment is residential
                
                area_bssh = bc["grossfloor_area"][bc_index_bssh - 1] * bssh["number_of_buildings"]
                #cooling_needs_bssh = bc["ued_cool"][bc_index_bssh - 1] * bssh["number_of_buildings"]
                heating_needs_bssh = bc["hwb_norm"][bc_index_bssh - 1] * area_bssh
                
                effective_indoor_temp_jan_bc = bc["effective_indoor_temp_jan"]
                effective_indoor_temp_jan_bssh = effective_indoor_temp_jan_bc[bc_index_bssh - 1]  #indoor temperatur in january of each building segment
                
                number_of_buildings_bssh = bssh["number_of_buildings"]
                uedh_sh_incl_bssh = bssh["uedh_sh_incl_all_service_factors_and_gains_dhw_losses"]

                #area_res_build_bssh = np.copy(area_bssh)
                #area_res_build_bssh[is_res_bssh == False] = 0       #set value of non-res buildings to Zero
                
                #average_indoor_temp_jan = np.sum(effective_indoor_temp_jan_bssh * area_res_build_bssh) / np.sum(area_res_build_bssh)
                average_indoor_temp_jan = np.sum(number_of_buildings_bssh * uedh_sh_incl_bssh * effective_indoor_temp_jan_bssh) / np.sum(number_of_buildings_bssh * uedh_sh_incl_bssh)
                
                #Get the uedh_sh share which indoor temp. is below a certain t_indoor_thresholds
                
                #t_indoor_thresholds = [17.5,18,18.5,19,19.5,20,20.5,21,21.5,22]     #List of temperature thresholds
                #t_indoor_thresholds = np.arange(average_indoor_temp_jan-2.5, average_indoor_temp_jan+3, 0.5)
                RESULTS = np.zeros(len(t_indoor_thresholds), dtype="f4")
                    
                tot_sum_uedh_sh = np.sum(uedh_sh_incl_bssh)
                for i, t_th in enumerate(t_indoor_thresholds):
                    is_below_temp_th = effective_indoor_temp_jan_bssh < t_th
                    share_of_uedh_sh_below_temp_th = np.sum(uedh_sh_incl_bssh[is_below_temp_th]) / tot_sum_uedh_sh
                    RESULTS[i] = share_of_uedh_sh_below_temp_th
                
                results_dict = {}
                counter = 0
                for i in thresh:
                    results_dict["Country"] = country
                    results_dict["Year"] = yr
                    results_dict["Scenario"] = scenario
                    results_dict["Weighted Average"] = average_indoor_temp_jan.round(2)
                    results_dict[i]=(RESULTS[counter]*100).round(2)
                    counter += 1
                 
                df_results = df_results.append(pd.DataFrame.from_dict(results_dict, orient='index').transpose())
                

                ##############################################
                print("Done in %4.2f sec"%(time.time() - st))
                #end of year loop
        
            hdf5_f.close()
            #end of hdf5 file loop
    
    print(time.time()-st0)
    print("  -----  THE END  --------")
    return df_results

In [80]:
country_codes=['SWE',
 'SVN',
 'SVK',
 'GRC',
 'FIN',
 'LTU',
 'MLT',
 'BEL',
 'IRL',
 'HRV',
 'LVA',
 'BGR',
 'NLD',
 'AUT',
 'PRT',
 'ESP',
 'FRA',
 'POL',
 'HUN',
 'DNK',
 'DEU',
 'LUX',
 'EST',
 'CYP',
 'CZE',
 'ROU',
 'ITA']

In [81]:
t_indoor_thresholds = np.arange(13,24,0.5)
thresh=t_indoor_thresholds.round(2)
col_names =  ['Country', 'Year', 'Scenario', 'Weighted Average']
col_names.extend(list(thresh))
final_results = pd.DataFrame(columns = col_names)

for country in country_codes:
    path = (r"projects/2021_ENEFIRST/invert/output_20210622/%s/" % country)
    files = os.listdir(path)
    
    for scenario in files:
        print(country)
        print(scenario)
        BASEDIR = "projects/2021_ENEFIRST/invert/output_20210622/%s/%s" % (country, scenario)
        #if scenario == '_scen_%s_electrification_ref_final_ab' % country.lower():
        #    continue
        temp_results = read_hdf5(BASEDIR, country, scenario)  
        final_results = final_results.append(temp_results)

SWE
_scen_swe_electrification_ref_final_install_iopt_inc_renov_1
Find files. This might take a while
1 files found in 0.01 sec
projects/2021_ENEFIRST/invert/output_20210622/SWE/_scen_swe_electrification_ref_final_install_iopt_inc_renov_1/001_buildings.hdf5
items stored in data container:
[('BC_2017', <HDF5 dataset "BC_2017": shape (470,), type "|V892">), ('BC_2050', <HDF5 dataset "BC_2050": shape (2395,), type "|V892">), ('BSDHW_2017', <HDF5 dataset "BSDHW_2017": shape (3111,), type "|V308">), ('BSDHW_2050', <HDF5 dataset "BSDHW_2050": shape (3138,), type "|V308">), ('BSSH_2017', <HDF5 dataset "BSSH_2017": shape (12871,), type "|V926">), ('BSSH_2050', <HDF5 dataset "BSSH_2050": shape (162905,), type "|V926">), ('BuildingCategories', <HDF5 dataset "BuildingCategories": shape (40,), type "|V198">), ('ResDefinition', <HDF5 dataset "ResDefinition": shape (45,), type "|V695">), ('ScenarioDefinition', <HDF5 dataset "ScenarioDefinition": shape (1,), type "|V3522">)]
Years stored in data conta

In [82]:
final_results

,Country,Year,Scenario,Weighted Average,13.0,13.5,14.0,14.5,15.0,15.5,...,20.5,21.0,21.5,22.0,22.5,23.0,23.5,24.0,24.5,25.0
0,SWE,2017,_scen_swe_electrification_ref_final_install_io...,18.17,0.0,0.0,0.0,0.0,0.0,0.0,...,85.4,88.83,90.32,95.23,100.0,100.0,100.0,100.0,100.0,100.0
0,SWE,2050,_scen_swe_electrification_ref_final_install_io...,19.870001,0.0,0.0,0.0,0.0,0.0,0.0,...,71.67,79.88,85.31,90.35,100.0,100.0,100.0,100.0,100.0,100.0
0,SWE,2017,_scen_swe_electrification_ref_final_install_io...,18.17,0.0,0.0,0.0,0.0,0.0,0.0,...,85.4,88.83,90.32,95.23,100.0,100.0,100.0,100.0,100.0,100.0
0,SWE,2050,_scen_swe_electrification_ref_final_install_io...,19.34,0.0,0.0,0.0,0.0,0.0,0.0,...,80.85,85.91,89.25,94.71,100.0,100.0,100.0,100.0,100.0,100.0
0,SWE,2017,_scen_swe_electrification_ref_final_install_io...,18.17,0.0,0.0,0.0,0.0,0.0,0.0,...,85.4,88.83,90.32,95.23,100.0,100.0,100.0,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,ITA,2050,_scen_ita_electrification_ref_final_install_io...,20.309999,0.0,0.0,0.0,0.0,0.0,0.0,...,48.7,59.12,87.9,95.91,100.0,100.0,100.0,100.0,100.0,100.0
0,ITA,2017,_scen_ita_electrification_ref_final_install_io...,19.040001,0.0,0.0,0.0,0.0,0.0,0.0,...,64.52,67.24,93.76,97.53,100.0,100.0,100.0,100.0,100.0,100.0
0,ITA,2050,_scen_ita_electrification_ref_final_install_io...,20.540001,0.0,0.0,0.0,0.0,0.0,0.0,...,42.65,56.28,85.93,95.23,100.0,100.0,100.0,100.0,100.0,100.0
0,ITA,2017,_scen_ita_electrification_ref_final_install_io...,19.040001,0.0,0.0,0.0,0.0,0.0,0.0,...,64.52,67.24,93.76,97.53,100.0,100.0,100.0,100.0,100.0,100.0


In [69]:
final_results.to_csv("final_results.csv")